In [1]:
import sys
sys.path.append('../')
import psml
import imp
import numpy as np
from os import listdir

In [2]:
"""Given a dataset, D, composed of n equally shaped state vector traces, 
the 'data dependent bits' are those that vary between invocations of a 
traced process, p. For example, if you have three binary traces of shape 
100x20, D.shape -> 3x100x20. If you sum along the 0th axis, you recover
a matrix, m, of shape 100x20 with elements in {0, 1, 2, 3}. Where m
is 0 or 3, the three traces agreed on their corresponding element, 0 or
1 respectively. Where m is 1 or 2, the evolution of a deterministic 
process must have depended on input or exogenous events. These are the 
'data dependent bits'. This view may be especially interesting for the
set of algorithms that don't branch conditionally based on data. In
that case, it's meaningful to compare the ith state across all traces
in D."""

"Given a dataset, D, composed of n equally shaped state vector traces, \nthe 'data dependent bits' are those that vary between invocations of a \ntraced process, p. For example, if you have three binary traces of shape \n100x20, D.shape -> 3x100x20. If you sum along the 0th axis, you recover\na matrix, m, of shape 100x20 with elements in {0, 1, 2, 3}. Where m\nis 0 or 3, the three traces agreed on their corresponding element, 0 or\n1 respectively. Where m is 1 or 2, the evolution of a deterministic \nprocess must have depended on input or exogenous events. These are the \n'data dependent bits'. This view may be especially interesting for the\nset of algorithms that don't branch conditionally based on data. In\nthat case, it's meaningful to compare the ith state across all traces\nin D."

In [3]:
#List of paths to traces
trace_dir = 'traces/'
l = [ trace_dir+i for i in listdir(trace_dir) ]
#Shorten for testing
l = l[:3]

#This is only done for mem reduction because we are going to convert
#The traces from byte to bit traces at a 8x cost in memory.
#The zero mask is a superset of the data dep mask.
zero_mask = psml.paths_to_zero_mask(l)

In [4]:
#Sanity check, the zero mask is truly a superset of the excited set, right?
excited_mask = psml.paths_to_excited_mask(l)
#If a set bit of b is not in a, the rhs will be smaller.
assert excited_mask.sum() == (zero_mask & excited_mask).sum()
print('success')

success


In [5]:
#Load byte traces masking with zero mask.
trcs = psml.paths_to_svmasked(l, zero_mask)
trcs.shape

(3, 454, 240)

In [6]:
#Expand excited byte traces to bit representation
trcs = psml.byte_to_bit_sv(trcs)
trcs.shape

(3, 454, 1920)

In [7]:
#Generate data dependent mask.
datadep_mask = psml.svarr_to_datadep_mask(trcs)
datadep_mask.shape, datadep_mask.sum()

((1920,), 17)

In [8]:
#Slice bit traces using data dependent mask.
trcs = psml.mask_svarr(trcs, datadep_mask)
trcs.shape

(3, 454, 17)

In [9]:
#Visualize the 
psml.show_arr(trcs[0])

In [10]:
#Sanity check, should have same result if done immediately on trace.
#This may fail unless l is restricted to a small set.
bt = psml.byte_to_bit_sv(psml.paths_to_tracearr(l))
dm = psml.svarr_to_datadep_mask(bt)
dbt = psml.mask_svarr(bt, dm)
psml.show_arr(dbt[0])

In [11]:
assert np.array_equal(trcs[0], dbt[0])
print("success")

success
